In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import ast # Để chuyển đổi chuỗi biểu diễn list/dict thành đối tượng Python

df = pd.read_csv('pd.csv')


# Xóa cột index không tên nếu có (dựa trên mô tả của bạn)
if df.columns[0].strip() == '':
    df = df.iloc[:, 1:]

print("Thông tin ban đầu về dữ liệu:")
print(df.info())
print("\n5 dòng đầu tiên:")
print(df.head())
print("\nThống kê mô tả cho các cột số:")
print(df.describe())
print("\nKiểm tra giá trị thiếu:")
print(df.isnull().sum())

Thông tin ban đầu về dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       1000 non-null   int64  
 1   parent_asin      1000 non-null   object 
 2   rating           1000 non-null   float64
 3   text             1000 non-null   object 
 4   user_id          1000 non-null   object 
 5   date             1000 non-null   object 
 6   title            1000 non-null   object 
 7   price            449 non-null    float64
 8   average_rating   1000 non-null   float64
 9   rating_number    1000 non-null   int64  
 10  categories       1000 non-null   object 
 11  features         1000 non-null   object 
 12  description      1000 non-null   object 
 13  main_category    957 non-null    object 
 14  store            997 non-null    object 
 15  sentiment_score  1000 non-null   int64  
dtypes: float64(3), int64(3), object

In [2]:
# 1. Chuyển đổi cột 'date' sang kiểu datetime
df['date'] = pd.to_datetime(df['date'])

# 2. Xử lý giá trị thiếu cho 'price'
# Ví dụ: điền bằng 0 hoặc giá trị trung bình/median.
# Một cách tiếp cận tốt hơn có thể là sử dụng một giá trị đặc biệt hoặc một cờ báo thiếu.
df['price'] = pd.to_numeric(df['price'], errors='coerce') # Đảm bảo là số, lỗi sẽ thành NaN
df['price'] = df['price'].fillna(0) # Điền NaN bằng 0 (ví dụ)

# 3. Đảm bảo các cột số khác là kiểu số
numerical_cols = ['rating', 'average_rating', 'rating_number', 'sentiment_score']
for col in numerical_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 4. Xử lý giá trị thiếu cho các cột quan trọng (ví dụ: loại bỏ dòng nếu 'rating' thiếu)
df.dropna(subset=['rating', 'parent_asin', 'date'], inplace=True)

# 5. Phân tích cú pháp các cột dạng chuỗi list (categories, features, description)
def parse_string_list_safe(s):
    if pd.isna(s):
        return []
    try:
        # Chuỗi description có vẻ như là list của một string JSON, rồi lại string
        # Cần kiểm tra định dạng cụ thể của cột này
        if isinstance(s, str) and s.startswith("[['") and s.endswith("']]"): # Dạng đặc biệt
             try:
                 # Thử loại bỏ các dấu ngoặc kép và ký tự escape không cần thiết
                 # Đây là phỏng đoán dựa trên dữ liệu mẫu, cần kiểm tra kỹ
                 s_cleaned = s.replace('\\"', '"') # Thay thế \" bằng "
                 parsed_outer_list = ast.literal_eval(s_cleaned)
                 if isinstance(parsed_outer_list, list) and len(parsed_outer_list) > 0:
                     # Nếu bên trong là một list các chuỗi JSON, ta có thể cần phân tích thêm
                     # Hiện tại, chỉ lấy chuỗi đầu tiên nếu nó là chuỗi
                     inner_content = parsed_outer_list[0]
                     if isinstance(inner_content, list) and len(inner_content) > 0 and isinstance(inner_content[0], str):
                         return [inner_content[0]] # Trả về list chứa chuỗi đó
                 return [] # Trả về list rỗng nếu không xử lý được
             except:
                 return [s] # Nếu vẫn lỗi, trả về chuỗi gốc trong list

        # Đối với categories và features
        evaluated_list = ast.literal_eval(s)
        if isinstance(evaluated_list, list):
            return evaluated_list
        else:
            return [str(evaluated_list)] # Nếu không phải list, biến nó thành list một phần tử
    except (ValueError, SyntaxError, TypeError):
        return [s] if isinstance(s, str) else [] # Nếu lỗi, trả về chuỗi gốc trong list hoặc list rỗng

df['categories_list'] = df['categories'].apply(parse_string_list_safe)
df['features_list'] = df['features'].apply(parse_string_list_safe)
# Cột description có vẻ phức tạp hơn, cần xem xét kỹ định dạng
# df['description_list'] = df['description'].apply(parse_string_list_safe)

# 6. Sắp xếp dữ liệu theo group_id (sản phẩm) và thời gian
df.sort_values(by=['parent_asin', 'date'], inplace=True)

# 7. Tạo 'time_idx': một chỉ số thời gian số nguyên liên tục cho mỗi nhóm
df['time_idx'] = df.groupby('parent_asin').cumcount()

print("\nKiểm tra giá trị thiếu sau tiền xử lý cơ bản:")
print(df.isnull().sum())
print("\n5 dòng sau khi tạo time_idx:")
print(df.shape)


Kiểm tra giá trị thiếu sau tiền xử lý cơ bản:
Unnamed: 0          0
parent_asin         0
rating              0
text                0
user_id             0
date                0
title               0
price               0
average_rating      0
rating_number       0
categories          0
features            0
description         0
main_category      43
store               3
sentiment_score     0
categories_list     0
features_list       0
time_idx            0
dtype: int64

5 dòng sau khi tạo time_idx:
(1000, 19)


In [3]:
# 1. Trích xuất các đặc trưng từ 'date' (time-varying known)
df['month'] = df['date'].dt.month.astype(str)
df['year'] = df['date'].dt.year.astype(str)
df['day_of_week'] = df['date'].dt.dayofweek.astype(str)
df['day_of_month'] = df['date'].dt.day.astype(str)
df['day_of_year'] = df['date'].dt.dayofyear.astype(float) # Dạng số thực
df['week_of_year'] = df['date'].dt.isocalendar().week.astype(str)


# 2. Xử lý các đặc trưng dạng list (categories_list, features_list)
# Ví dụ đơn giản: lấy phần tử đầu tiên làm đại diện hoặc tạo đặc trưng one-hot/multi-hot
df['main_cat_from_list'] = df['categories_list'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else 'Unknown')

# 3. Các đặc trưng văn bản:
# Hiện tại, chúng ta có 'sentiment_score'.
# Nếu muốn phức tạp hơn, bạn có thể:
# - Sử dụng TF-IDF cho 'text', 'title'.
# - Sử dụng word/sentence embeddings (ví dụ: SentenceBERT) cho 'text', 'title', 'features', 'description'.
#   Việc này sẽ tạo ra các vector số thực, có thể được thêm vào `time_varying_unknown_reals`.
#   Ví dụ (khái niệm):
#   from sentence_transformers import SentenceTransformer
#   model_st = SentenceTransformer('all-MiniLM-L6-v2')
#   df['text_embedding'] = df['text'].apply(lambda x: model_st.encode(str(x)))
#   # Sau đó, bạn cần tách embedding thành nhiều cột hoặc xử lý nó dưới dạng một vector.

# 4. Chuyển đổi các cột categorical sang kiểu 'category' của Pandas
# (PyTorch Forecasting sẽ xử lý chúng, nhưng việc khai báo rõ ràng là tốt)
categorical_cols_static = ['parent_asin', 'store', 'main_category', 'main_cat_from_list']
categorical_cols_time_varying = ['month', 'year', 'day_of_week', 'day_of_month', 'week_of_year']

for col in categorical_cols_static + categorical_cols_time_varying:
    if col in df.columns:
        df[col] = df[col].astype(str).astype('category')

print("\n5 dòng sau khi thêm đặc trưng thời gian:")
print(df[['date', 'time_idx', 'month', 'year', 'day_of_week', 'main_cat_from_list']].head())


5 dòng sau khi thêm đặc trưng thời gian:
        date  time_idx month  year day_of_week         main_cat_from_list
0 2015-07-24         0     7  2015           4  Clothing, Shoes & Jewelry
1 2014-10-24         0    10  2014           4  Clothing, Shoes & Jewelry
2 2016-04-01         0     4  2016           4  Clothing, Shoes & Jewelry
3 2012-10-06         0    10  2012           5  Clothing, Shoes & Jewelry
4 2017-06-09         1     6  2017           4  Clothing, Shoes & Jewelry


In [4]:

from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer, NaNLabelEncoder
from pytorch_forecasting.metrics import MAE, SMAPE, QuantileLoss
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

import pytorch_forecasting


# Xác định các tham số cho TimeSeriesDataSet
# max_encoder_length: số lượng bản ghi quá khứ mà mô hình sẽ xem xét.
# max_prediction_length: số lượng bản ghi tương lai mà mô hình sẽ dự đoán.
# Vì chúng ta dự đoán rating cho mỗi review, có thể prediction_length = 1.
# Tuy nhiên, TFT được thiết kế để dự đoán một chuỗi, nên chúng ta có thể giữ nó > 1
# và chỉ quan tâm đến dự đoán đầu tiên, hoặc dự đoán rating trung bình trong khoảng đó.
# Hãy bắt đầu với việc dự đoán rating tại bước thời gian tiếp theo.
max_encoder_length = 2  # Ví dụ: xem xét 30 đánh giá trước đó của sản phẩm
max_prediction_length = 1 # Dự đoán rating cho đánh giá tiếp theo

# Lọc bỏ các chuỗi thời gian quá ngắn
min_series_length = max_encoder_length + max_prediction_length
series_lengths = df.groupby('parent_asin')['time_idx'].count()
valid_series_ids = series_lengths[series_lengths >= min_series_length].index
df_filtered = df[df['parent_asin'].isin(valid_series_ids)].copy()

if df_filtered.empty:
    print(f"Không có chuỗi thời gian nào đủ dài cho max_encoder_length={max_encoder_length} "
          f"và max_prediction_length={max_prediction_length}. Hãy thử giảm các giá trị này.")
    exit()
else:
    print(f"Số dòng sau khi lọc các chuỗi ngắn: {df_filtered.shape[0]}")


# Xác định điểm cắt cho tập huấn luyện và kiểm định
# Ví dụ: sử dụng 80% dữ liệu (theo time_idx tổng thể hoặc theo từng nhóm) cho huấn luyện
# Cách tiếp cận tốt là chia theo một mốc thời gian cụ thể.
# Ở đây, chúng ta sẽ chia theo `time_idx` cuối cùng.
training_cutoff = df_filtered["time_idx"].max() - max_prediction_length * 5 # Giữ lại 5*prediction_length cho validation

# Xử lý NaN trong target một cách cẩn thận trước khi tạo dataset
# print(f"Số NaN trong cột rating trước khi tạo dataset: {df_filtered['rating'].isnull().sum()}")
# df_filtered.dropna(subset=['rating'], inplace=True) # Rất quan trọng

# Các cột đặc trưng
static_categoricals = ['store', 'main_category', 'main_cat_from_list'] # parent_asin là group_id
# static_reals = [] # Nếu có

time_varying_known_categoricals = ['month', 'year', 'day_of_week', 'day_of_month', 'week_of_year']
time_varying_known_reals = ['day_of_year'] # `price` có thể là known nếu được đặt trước

time_varying_unknown_reals = ['price', 'average_rating', 'rating_number', 'sentiment_score']
# Nếu bạn có text embeddings (dạng vector số thực), chúng sẽ vào đây.
# time_varying_unknown_categoricals = [] # Nếu có

# Đảm bảo các cột này tồn tại trong df_filtered và có kiểu dữ liệu phù hợp
cols_to_check = static_categoricals + \
                time_varying_known_categoricals + time_varying_known_reals + \
                time_varying_unknown_reals + ['parent_asin', 'rating', 'time_idx']

for col in cols_to_check:
    if col not in df_filtered.columns:
        print(f"CẢNH BÁO: Cột '{col}' không tồn tại trong DataFrame!")
        # Xử lý bằng cách loại bỏ khỏi danh sách hoặc tạo cột giả
        if col in static_categoricals: static_categoricals.remove(col)
        # ... tương tự cho các danh sách khác

# Loại bỏ các cột không có trong df_filtered khỏi danh sách đặc trưng
static_categoricals = [col for col in static_categoricals if col in df_filtered.columns]
time_varying_known_categoricals = [col for col in time_varying_known_categoricals if col in df_filtered.columns]
time_varying_known_reals = [col for col in time_varying_known_reals if col in df_filtered.columns]
time_varying_unknown_reals = [col for col in time_varying_unknown_reals if col in df_filtered.columns]


try:
    training_dataset = TimeSeriesDataSet(
        df_filtered[lambda x: x.time_idx <= training_cutoff],
        time_idx="time_idx",
        target="rating",
        group_ids=["parent_asin"],
        max_encoder_length=max_encoder_length,
        max_prediction_length=max_prediction_length,
        min_prediction_length=max_prediction_length,
        static_categoricals=static_categoricals,
        # static_reals=static_reals,
        time_varying_known_categoricals=time_varying_known_categoricals,
        time_varying_known_reals=time_varying_known_reals,
        time_varying_unknown_reals=time_varying_unknown_reals,
        # time_varying_unknown_categoricals=time_varying_unknown_categoricals,
        target_normalizer=GroupNormalizer(groups=["parent_asin"]),

        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True,
        categorical_encoders={col: NaNLabelEncoder(add_nan=True) for col in static_categoricals + time_varying_known_categoricals if col in df_filtered.columns}
    )

    # Tạo tập kiểm định (validation set)
    # Đảm bảo dữ liệu kiểm định bắt đầu sau dữ liệu huấn luyện
    validation_dataset = TimeSeriesDataSet.from_dataset(
        training_dataset, # Kế thừa các encoders, scaling từ tập training
        df_filtered[lambda x: x.time_idx > training_cutoff],
        min_prediction_idx=training_cutoff + 1, # Bắt đầu dự đoán sau điểm cắt
        stop_randomization=True # Quan trọng cho tập kiểm định
    )

    # Tạo Dataloaders
    batch_size = 128  # Điều chỉnh dựa trên bộ nhớ của bạn
    # num_workers=0 nếu chạy trên Windows hoặc Jupyter notebook để tránh lỗi
    train_dataloader = training_dataset.to_dataloader(train=True, batch_size=batch_size, num_workers=4)
    val_dataloader = validation_dataset.to_dataloader(train=False, batch_size=batch_size * 2, num_workers=4)
    
    train_dataloader.pin_memory = True
    val_dataloader.pin_memory = True
    print("TimeSeriesDataSet và DataLoaders đã được tạo thành công.")

except Exception as e:
    print(f"Lỗi khi tạo TimeSeriesDataSet hoặc Dataloaders: {e}")
    import traceback
    traceback.print_exc()
    print("Kiểm tra các vấn đề sau:")
    print("- NaN trong các cột quan trọng (target, group_ids, time_idx).")
    print("- Các đặc trưng hạng mục có được mã hóa đúng cách hoặc là kiểu chuỗi không.")
    print("- Tất cả các cột đặc trưng được chỉ định có tồn tại trong DataFrame và có kiểu nhất quán không.")
    print("- Độ dài chuỗi thời gian có đủ không.")
    exit()

c:\Users\PC\anaconda3\envs\DL_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Số dòng sau khi lọc các chuỗi ngắn: 335
TimeSeriesDataSet và DataLoaders đã được tạo thành công.


C:\Users\PC\AppData\Local\Temp\ipykernel_22144\2600318772.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  series_lengths = df.groupby('parent_asin')['time_idx'].count()


In [5]:
pl.seed_everything(42) # Để có thể tái tạo kết quả

# Định nghĩa mô hình TFT
# Điều chỉnh các siêu tham số như hidden_size, lstm_layers, num_heads, dropout
# dựa trên kích thước và độ phức tạp của bộ dữ liệu.
tft = TemporalFusionTransformer.from_dataset(
    training_dataset,
    learning_rate=0.001, # Tốc độ học phổ biến: 0.001 đến 0.1
    hidden_size=32,        # Kích thước lớp ẩn (ví dụ: 16, 32, 64)
    attention_head_size=4, # Số lượng attention heads
    dropout=0.1,           # Tỷ lệ dropout
    hidden_continuous_size=16, # Kích thước lớp ẩn cho biến liên tục
    output_size=7,         # Số lượng quantiles để dự đoán (7 cho QuantileLoss mặc định)
                           # Nếu bạn muốn dự đoán một giá trị cụ thể, có thể đặt loss là MAE() hoặc SMAPE()
                           # và output_size=1, nhưng TFT thường được thiết lập cho quantiles.
    loss=QuantileLoss(),   # Hàm mất mát (QuantileLoss, MAE, SMAPE)
    # reduce_on_plateau_patience=4 # Cho learning rate scheduler
)
print(f"Số lượng tham số trong mạng: {tft.size()/1e3:.1f}k")

Seed set to 42


Số lượng tham số trong mạng: 80.0k


c:\Users\PC\anaconda3\envs\DL_env\lib\site-packages\lightning\pytorch\utilities\parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
c:\Users\PC\anaconda3\envs\DL_env\lib\site-packages\lightning\pytorch\utilities\parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


In [6]:
print(f"PyTorch version: {torch.__version__}")
print(f"PyTorch Lightning version: {pl.__version__}")
print(f"Pytorch Forecasting version: {pytorch_forecasting.__version__}")

PyTorch version: 2.6.0+cu126
PyTorch Lightning version: 2.5.1.post0
Pytorch Forecasting version: 1.3.0


In [7]:
print(f"Type of tft object: {type(tft)}")
print(f"Is tft a LightningModule? {isinstance(tft, pl.LightningModule)}")
print(f"Is tft a TemporalFusionTransformer from pytorch_forecasting? {isinstance(tft, TemporalFusionTransformer)}")


Type of tft object: <class 'pytorch_forecasting.models.temporal_fusion_transformer._tft.TemporalFusionTransformer'>
Is tft a LightningModule? False
Is tft a TemporalFusionTransformer from pytorch_forecasting? True


In [8]:
for batch in train_dataloader:
    print("Kiểm tra batch từ train_dataloader (dạng tuple):")
    if isinstance(batch, tuple):
        for i, tensor in enumerate(batch):
            if isinstance(tensor, torch.Tensor):
                print(f"  Tensor tại index {i} có device: {tensor.device}")
            else:
                print(f"  Phần tử tại index {i} không phải là tensor (type: {type(tensor)})")
    else:
        print("  Batch không phải là tuple.")
    break

for batch in val_dataloader:
    print("\nKiểm tra batch từ val_dataloader (dạng tuple):")
    if isinstance(batch, tuple):
        for i, tensor in enumerate(batch):
            if isinstance(tensor, torch.Tensor):
                print(f"  Tensor tại index {i} có device: {tensor.device}")
            else:
                print(f"  Phần tử tại index {i} không phải là tensor (type: {type(tensor)})")
    else:
        print("  Batch không phải là tuple.")
    break

Kiểm tra batch từ train_dataloader (dạng tuple):
  Batch không phải là tuple.

Kiểm tra batch từ val_dataloader (dạng tuple):
  Batch không phải là tuple.


In [9]:
for batch in train_dataloader:
    print("Kiểm tra tensors trong batch từ train_dataloader:")
    if isinstance(batch, tuple) and isinstance(batch[0], dict):
        input_dict = batch[0]
        for key, value in input_dict.items():
            if isinstance(value, torch.Tensor):
                print(f"  Tensor '{key}' có device: {value.device}")
            else:
                print(f"  '{key}' không phải là tensor (type: {type(value)})")
    else:
        print("  Cấu trúc batch không như mong đợi cho train_dataloader.")
    break

for batch in val_dataloader:
    print("\nKiểm tra tensors trong batch từ val_dataloader:")
    if isinstance(batch, tuple) and isinstance(batch[0], dict):
        input_dict = batch[0]
        for key, value in input_dict.items():
            if isinstance(value, torch.Tensor):
                print(f"  Tensor '{key}' có device: {value.device}")
            else:
                print(f"  '{key}' không phải là tensor (type: {type(value)})")
    else:
        print("  Cấu trúc batch không như mong đợi cho val_dataloader.")
    break

Kiểm tra tensors trong batch từ train_dataloader:
  Cấu trúc batch không như mong đợi cho train_dataloader.

Kiểm tra tensors trong batch từ val_dataloader:
  Cấu trúc batch không như mong đợi cho val_dataloader.


In [10]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_forecasting import TemporalFusionTransformer

# Giả sử bạn đã có train_dataloader và val_dataloader


# Định nghĩa một LightningModule bao bọc mô hình TFT
class TFTLightningModule(pl.LightningModule):
    def __init__(self, tft_model, learning_rate):
        super().__init__()
        self.tft_model = tft_model
        self.learning_rate = learning_rate

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.tft_model(x)
        loss = self.loss_fn(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.tft_model(x)
        loss = self.loss_fn(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def loss_fn(self, y_hat, y):
        # Định nghĩa hàm loss phù hợp với đầu ra của mô hình và nhãn
        # Ví dụ: Mean Squared Error (MSE)
        return torch.mean((y_hat - y)**2)

# Cấu hình Trainer của PyTorch Lightning
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor() # Theo dõi tốc độ học

# Kiểm tra xem GPU có sẵn không
accelerator = "gpu" if torch.cuda.is_available() else "cpu"
devices = 1 if torch.cuda.is_available() else None

trainer = pl.Trainer(
    max_epochs=5, # Bắt đầu với số lượng epochs nhỏ để kiểm tra pipeline (ví dụ: 3-10)
                  # Tăng lên sau (ví dụ: 50-100)
    accelerator=accelerator,
    devices=devices,
    gradient_clip_val=0.1, # Giúp ổn định quá trình huấn luyện
    limit_train_batches=50,  # Giới hạn số batch cho mỗi epoch để lặp nhanh hơn trong quá trình phát triển
    limit_val_batches=20,
    callbacks=[lr_logger, early_stop_callback],
    # logger=pl.loggers.TensorBoardLogger("lightning_logs") # Để sử dụng TensorBoard
)

# Khởi tạo LightningModule với mô hình tft và learning rate
learning_rate = 0.001 # Điều chỉnh learning rate nếu cần
tft_lightning = TFTLightningModule(tft, learning_rate)

# Huấn luyện mô hình
try:
    print("Bắt đầu huấn luyện mô hình...")
    trainer.fit(
        tft_lightning,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
    )
    print("Hoàn thành huấn luyện mô hình.")
except Exception as e:
    print(f"Đã xảy ra lỗi trong quá trình huấn luyện: {e}")
    import traceback
    traceback.print_exc()
    exit()

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                      | Params | Mode 
----------------------------------------------------------------
0 | tft_model | TemporalFusionTransformer | 80.0 K | train
----------------------------------------------------------------
80.0 K    Trainable params
0         Non-trainable params
80.0 K    Total params
0.320     Total estimated model params size (MB)
385       Modules in train mode
0         Modules in eval mode


Bắt đầu huấn luyện mô hình...
Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\PC\anaconda3\envs\DL_env\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]Đã xảy ra lỗi trong quá trình huấn luyện: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument tensors in method wrapper_CUDA_cat)


Traceback (most recent call last):
  File "C:\Users\PC\AppData\Local\Temp\ipykernel_22144\1721587369.py", line 66, in <module>
    trainer.fit(
  File "c:\Users\PC\anaconda3\envs\DL_env\lib\site-packages\pytorch_lightning\trainer\trainer.py", line 561, in fit
    call._call_and_handle_interrupt(
  File "c:\Users\PC\anaconda3\envs\DL_env\lib\site-packages\pytorch_lightning\trainer\call.py", line 48, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
  File "c:\Users\PC\anaconda3\envs\DL_env\lib\site-packages\pytorch_lightning\trainer\trainer.py", line 599, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "c:\Users\PC\anaconda3\envs\DL_env\lib\site-packages\pytorch_lightning\trainer\trainer.py", line 1012, in _run
    results = self._run_stage()
  File "c:\Users\PC\anaconda3\envs\DL_env\lib\site-packages\pytorch_lightning\trainer\trainer.py", line 1054, in _run_stage
    self._run_sanity_check()
  File "c:\Users\PC\anaconda3\envs\DL_env\lib\site-packages\py

In [ ]:
try:
    # Tải mô hình tốt nhất từ checkpoint
    best_model_path = trainer.checkpoint_callback.best_model_path
    if best_model_path:
        print(f"Đang tải mô hình tốt nhất từ: {best_model_path}")
        best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

        # Đánh giá trên tập kiểm định
        actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
        raw_predictions = best_tft.predict(val_dataloader, mode="raw", return_x=True)
        predictions = raw_predictions.output # Lấy phần output của dự đoán

        # Nếu sử dụng QuantileLoss, predictions sẽ có dạng [batch_size, prediction_length, num_quantiles]
        # Để có một dự đoán điểm, bạn có thể lấy quantile giữa (thường là median).
        if tft.loss.quantiles:
            median_prediction_index = len(tft.loss.quantiles) // 2
            point_predictions = predictions[:, :, median_prediction_index]
        else: # Nếu loss là MAE hoặc SMAPE, predictions đã là dự đoán điểm
            point_predictions = predictions.squeeze(-1) # Bỏ chiều cuối nếu nó là 1

        # Tính toán MAE (Mean Absolute Error)
        mae_value = (actuals - point_predictions).abs().mean()
        print(f"Validation MAE (trên dự đoán điểm): {mae_value.item()}")

        # Bạn cũng có thể tính các độ đo khác như RMSE, SMAPE tùy thuộc vào hàm loss đã chọn.

        # Xem một vài dự đoán mẫu
        print("\nDự đoán mẫu (5 dự đoán đầu tiên):")
        for i in range(min(5, len(actuals))):
            print(f"Thực tế: {actuals[i].item():.2f}, Dự đoán: {point_predictions[i].item():.2f}")

        # Trực quan hóa dự đoán (tùy chọn, cần matplotlib)
        # import matplotlib.pyplot as plt
        # sample_idx = 0 # Chọn một mẫu để vẽ
        # true_values = raw_predictions.x['decoder_target'][sample_idx] # Giá trị thực tế trong khoảng dự đoán
        # predicted_values = point_predictions[sample_idx]
        # past_values = raw_predictions.x['encoder_target'][sample_idx] # Giá trị trong encoder (quá khứ)

        # plt.figure(figsize=(12, 6))
        # time_steps_past = np.arange(-len(past_values), 0)
        # time_steps_future = np.arange(0, len(true_values))

        # plt.plot(time_steps_past, past_values, label="Lịch sử Rating (Encoder)")
        # plt.plot(time_steps_future, true_values, label="Rating Thực tế (Decoder)")
        # plt.plot(time_steps_future, predicted_values, label="Rating Dự đoán")
        # plt.title(f"Dự đoán Rating cho một sản phẩm (mẫu {sample_idx})")
        # plt.xlabel("Bước thời gian (tương đối)")
        # plt.ylabel("Rating")
        # plt.legend()
        # plt.show()

    else:
        print("Không tìm thấy checkpoint mô hình tốt nhất. Có thể quá trình huấn luyện chưa tạo checkpoint.")

except Exception as e:
    print(f"Đã xảy ra lỗi trong quá trình đánh giá hoặc dự đoán: {e}")
    import traceback
    traceback.print_exc()

Không tìm thấy checkpoint mô hình tốt nhất. Có thể quá trình huấn luyện chưa tạo checkpoint.


: 